In [ ]:
# Code:
# https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/acgan/acgan.py

In [5]:
# Imports
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [8]:
# Configs

n_epochs=200
batch_size=64
lr=0.0002
b1=0.5
b2=0.999
latent_dim=100
n_classes=10
img_size=32
channels=1
sample_interval=400


In [9]:
# Functions and Model
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


In [10]:
# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

cuda = True

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [12]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

0.1%

100.1%

Extracting ../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw


0.5%5%

Extracting ../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw


180.4%

Extracting ../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw
Extracting ../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw
Processing...
Done!


In [15]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "../images/%d.png" % batches_done, nrow=n_row, normalize=True)
    

In [16]:
# Training
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)


[Epoch 0/200] [Batch 0/938] [D loss: 1.497853, acc: 9%] [G loss: 1.491239]
[Epoch 0/200] [Batch 1/938] [D loss: 1.497986, acc: 7%] [G loss: 1.491616]
[Epoch 0/200] [Batch 2/938] [D loss: 1.497748, acc: 14%] [G loss: 1.491213]
[Epoch 0/200] [Batch 3/938] [D loss: 1.497929, acc: 7%] [G loss: 1.491696]
[Epoch 0/200] [Batch 4/938] [D loss: 1.497744, acc: 5%] [G loss: 1.491741]
[Epoch 0/200] [Batch 5/938] [D loss: 1.497814, acc: 9%] [G loss: 1.491944]
[Epoch 0/200] [Batch 6/938] [D loss: 1.497926, acc: 7%] [G loss: 1.492474]
[Epoch 0/200] [Batch 7/938] [D loss: 1.497587, acc: 14%] [G loss: 1.492329]
[Epoch 0/200] [Batch 8/938] [D loss: 1.497893, acc: 11%] [G loss: 1.492996]
[Epoch 0/200] [Batch 9/938] [D loss: 1.497888, acc: 7%] [G loss: 1.493207]
[Epoch 0/200] [Batch 10/938] [D loss: 1.497869, acc: 8%] [G loss: 1.493077]
[Epoch 0/200] [Batch 11/938] [D loss: 1.497617, acc: 7%] [G loss: 1.493303]
[Epoch 0/200] [Batch 12/938] [D loss: 1.497687, acc: 8%] [G loss: 1.493407]
[Epoch 0/200] [Batc

[Epoch 0/200] [Batch 108/938] [D loss: 1.498500, acc: 10%] [G loss: 1.505882]
[Epoch 0/200] [Batch 109/938] [D loss: 1.499792, acc: 8%] [G loss: 1.501962]
[Epoch 0/200] [Batch 110/938] [D loss: 1.500608, acc: 7%] [G loss: 1.500743]
[Epoch 0/200] [Batch 111/938] [D loss: 1.500042, acc: 13%] [G loss: 1.499482]
[Epoch 0/200] [Batch 112/938] [D loss: 1.499197, acc: 10%] [G loss: 1.499197]
[Epoch 0/200] [Batch 113/938] [D loss: 1.499562, acc: 7%] [G loss: 1.499289]
[Epoch 0/200] [Batch 114/938] [D loss: 1.496945, acc: 10%] [G loss: 1.499475]
[Epoch 0/200] [Batch 115/938] [D loss: 1.495512, acc: 11%] [G loss: 1.499715]
[Epoch 0/200] [Batch 116/938] [D loss: 1.494918, acc: 10%] [G loss: 1.497970]
[Epoch 0/200] [Batch 117/938] [D loss: 1.495335, acc: 13%] [G loss: 1.495018]
[Epoch 0/200] [Batch 118/938] [D loss: 1.495097, acc: 12%] [G loss: 1.490754]
[Epoch 0/200] [Batch 119/938] [D loss: 1.496156, acc: 6%] [G loss: 1.488932]
[Epoch 0/200] [Batch 120/938] [D loss: 1.495815, acc: 5%] [G loss: 1

[Epoch 0/200] [Batch 218/938] [D loss: 1.496170, acc: 19%] [G loss: 1.500919]
[Epoch 0/200] [Batch 219/938] [D loss: 1.495437, acc: 20%] [G loss: 1.498939]
[Epoch 0/200] [Batch 220/938] [D loss: 1.496962, acc: 16%] [G loss: 1.499902]
[Epoch 0/200] [Batch 221/938] [D loss: 1.495389, acc: 14%] [G loss: 1.498770]
[Epoch 0/200] [Batch 222/938] [D loss: 1.497842, acc: 16%] [G loss: 1.499172]
[Epoch 0/200] [Batch 223/938] [D loss: 1.496120, acc: 16%] [G loss: 1.496592]
[Epoch 0/200] [Batch 224/938] [D loss: 1.497908, acc: 15%] [G loss: 1.495459]
[Epoch 0/200] [Batch 225/938] [D loss: 1.494958, acc: 18%] [G loss: 1.498078]
[Epoch 0/200] [Batch 226/938] [D loss: 1.496416, acc: 19%] [G loss: 1.492516]
[Epoch 0/200] [Batch 227/938] [D loss: 1.497437, acc: 23%] [G loss: 1.490687]
[Epoch 0/200] [Batch 228/938] [D loss: 1.495046, acc: 25%] [G loss: 1.494759]
[Epoch 0/200] [Batch 229/938] [D loss: 1.497625, acc: 19%] [G loss: 1.492430]
[Epoch 0/200] [Batch 230/938] [D loss: 1.494865, acc: 22%] [G lo

[Epoch 0/200] [Batch 328/938] [D loss: 1.409642, acc: 40%] [G loss: 1.519229]
[Epoch 0/200] [Batch 329/938] [D loss: 1.403790, acc: 32%] [G loss: 1.505138]
[Epoch 0/200] [Batch 330/938] [D loss: 1.402722, acc: 42%] [G loss: 1.491199]
[Epoch 0/200] [Batch 331/938] [D loss: 1.411333, acc: 37%] [G loss: 1.465479]
[Epoch 0/200] [Batch 332/938] [D loss: 1.405832, acc: 38%] [G loss: 1.479071]
[Epoch 0/200] [Batch 333/938] [D loss: 1.417900, acc: 30%] [G loss: 1.466012]
[Epoch 0/200] [Batch 334/938] [D loss: 1.398239, acc: 37%] [G loss: 1.445237]
[Epoch 0/200] [Batch 335/938] [D loss: 1.364596, acc: 44%] [G loss: 1.434312]
[Epoch 0/200] [Batch 336/938] [D loss: 1.382176, acc: 40%] [G loss: 1.421601]
[Epoch 0/200] [Batch 337/938] [D loss: 1.362144, acc: 41%] [G loss: 1.437453]
[Epoch 0/200] [Batch 338/938] [D loss: 1.359351, acc: 45%] [G loss: 1.422779]
[Epoch 0/200] [Batch 339/938] [D loss: 1.369110, acc: 39%] [G loss: 1.452441]
[Epoch 0/200] [Batch 340/938] [D loss: 1.353384, acc: 39%] [G lo

[Epoch 0/200] [Batch 438/938] [D loss: 1.320638, acc: 52%] [G loss: 1.411770]
[Epoch 0/200] [Batch 439/938] [D loss: 1.312742, acc: 55%] [G loss: 1.443708]
[Epoch 0/200] [Batch 440/938] [D loss: 1.308216, acc: 57%] [G loss: 1.382076]
[Epoch 0/200] [Batch 441/938] [D loss: 1.310164, acc: 53%] [G loss: 1.402252]
[Epoch 0/200] [Batch 442/938] [D loss: 1.319458, acc: 53%] [G loss: 1.392408]
[Epoch 0/200] [Batch 443/938] [D loss: 1.323328, acc: 53%] [G loss: 1.428720]
[Epoch 0/200] [Batch 444/938] [D loss: 1.280651, acc: 60%] [G loss: 1.369097]
[Epoch 0/200] [Batch 445/938] [D loss: 1.314538, acc: 50%] [G loss: 1.409153]
[Epoch 0/200] [Batch 446/938] [D loss: 1.294941, acc: 55%] [G loss: 1.395123]
[Epoch 0/200] [Batch 447/938] [D loss: 1.283681, acc: 62%] [G loss: 1.350698]
[Epoch 0/200] [Batch 448/938] [D loss: 1.292718, acc: 57%] [G loss: 1.426707]
[Epoch 0/200] [Batch 449/938] [D loss: 1.299609, acc: 54%] [G loss: 1.403779]
[Epoch 0/200] [Batch 450/938] [D loss: 1.350353, acc: 48%] [G lo

...

[Epoch 9/200] [Batch 321/938] [D loss: 1.067906, acc: 98%] [G loss: 1.108505]
[Epoch 9/200] [Batch 322/938] [D loss: 1.102483, acc: 95%] [G loss: 1.075915]
[Epoch 9/200] [Batch 323/938] [D loss: 1.102683, acc: 94%] [G loss: 1.132824]
[Epoch 9/200] [Batch 324/938] [D loss: 1.083328, acc: 97%] [G loss: 1.091440]
[Epoch 9/200] [Batch 325/938] [D loss: 1.074704, acc: 97%] [G loss: 1.187520]
[Epoch 9/200] [Batch 326/938] [D loss: 1.083032, acc: 99%] [G loss: 1.028088]
[Epoch 9/200] [Batch 327/938] [D loss: 1.084313, acc: 94%] [G loss: 1.103103]
[Epoch 9/200] [Batch 328/938] [D loss: 1.098739, acc: 97%] [G loss: 1.140667]
[Epoch 9/200] [Batch 329/938] [D loss: 1.094163, acc: 100%] [G loss: 1.108829]
[Epoch 9/200] [Batch 330/938] [D loss: 1.047839, acc: 96%] [G loss: 1.096711]
[Epoch 9/200] [Batch 331/938] [D loss: 1.090537, acc: 96%] [G loss: 1.132466]
[Epoch 9/200] [Batch 332/938] [D loss: 1.068974, acc: 96%] [G loss: 1.075137]
[Epoch 9/200] [Batch 333/938] [D loss: 1.095346, acc: 96%] [G l

[Epoch 9/200] [Batch 426/938] [D loss: 1.085093, acc: 96%] [G loss: 1.151905]
[Epoch 9/200] [Batch 427/938] [D loss: 1.103019, acc: 96%] [G loss: 1.164798]
[Epoch 9/200] [Batch 428/938] [D loss: 1.085092, acc: 96%] [G loss: 1.124659]
[Epoch 9/200] [Batch 429/938] [D loss: 1.094836, acc: 96%] [G loss: 1.047928]
[Epoch 9/200] [Batch 430/938] [D loss: 1.101620, acc: 97%] [G loss: 1.077176]
[Epoch 9/200] [Batch 431/938] [D loss: 1.104989, acc: 98%] [G loss: 1.083390]
[Epoch 9/200] [Batch 432/938] [D loss: 1.070819, acc: 97%] [G loss: 1.128084]
[Epoch 9/200] [Batch 433/938] [D loss: 1.089094, acc: 96%] [G loss: 1.090495]
[Epoch 9/200] [Batch 434/938] [D loss: 1.122532, acc: 98%] [G loss: 1.097129]
[Epoch 9/200] [Batch 435/938] [D loss: 1.080924, acc: 96%] [G loss: 1.105747]
[Epoch 9/200] [Batch 436/938] [D loss: 1.083529, acc: 96%] [G loss: 1.134473]
[Epoch 9/200] [Batch 437/938] [D loss: 1.123328, acc: 95%] [G loss: 1.108833]
[Epoch 9/200] [Batch 438/938] [D loss: 1.130277, acc: 93%] [G lo

[Epoch 9/200] [Batch 536/938] [D loss: 1.109717, acc: 96%] [G loss: 1.128879]
[Epoch 9/200] [Batch 537/938] [D loss: 1.111459, acc: 98%] [G loss: 1.118430]
[Epoch 9/200] [Batch 538/938] [D loss: 1.113173, acc: 95%] [G loss: 1.064378]
[Epoch 9/200] [Batch 539/938] [D loss: 1.117844, acc: 95%] [G loss: 1.141148]
[Epoch 9/200] [Batch 540/938] [D loss: 1.108821, acc: 97%] [G loss: 1.140903]
[Epoch 9/200] [Batch 541/938] [D loss: 1.088153, acc: 97%] [G loss: 1.114690]
[Epoch 9/200] [Batch 542/938] [D loss: 1.063594, acc: 96%] [G loss: 1.100572]
[Epoch 9/200] [Batch 543/938] [D loss: 1.077328, acc: 99%] [G loss: 1.116255]
[Epoch 9/200] [Batch 544/938] [D loss: 1.088593, acc: 97%] [G loss: 1.097299]
[Epoch 9/200] [Batch 545/938] [D loss: 1.105308, acc: 97%] [G loss: 1.112789]
[Epoch 9/200] [Batch 546/938] [D loss: 1.109605, acc: 97%] [G loss: 1.093323]
[Epoch 9/200] [Batch 547/938] [D loss: 1.070852, acc: 98%] [G loss: 1.121828]
[Epoch 9/200] [Batch 548/938] [D loss: 1.103852, acc: 95%] [G lo

[Epoch 9/200] [Batch 646/938] [D loss: 1.089566, acc: 98%] [G loss: 1.100420]
[Epoch 9/200] [Batch 647/938] [D loss: 1.080558, acc: 97%] [G loss: 1.078786]
[Epoch 9/200] [Batch 648/938] [D loss: 1.088964, acc: 94%] [G loss: 1.130198]
[Epoch 9/200] [Batch 649/938] [D loss: 1.078973, acc: 96%] [G loss: 1.121805]
[Epoch 9/200] [Batch 650/938] [D loss: 1.074155, acc: 97%] [G loss: 1.129241]
[Epoch 9/200] [Batch 651/938] [D loss: 1.096089, acc: 96%] [G loss: 1.070014]
[Epoch 9/200] [Batch 652/938] [D loss: 1.090037, acc: 100%] [G loss: 1.130458]
[Epoch 9/200] [Batch 653/938] [D loss: 1.123326, acc: 92%] [G loss: 1.088299]
[Epoch 9/200] [Batch 654/938] [D loss: 1.102554, acc: 96%] [G loss: 1.103059]
[Epoch 9/200] [Batch 655/938] [D loss: 1.126745, acc: 96%] [G loss: 1.095302]
[Epoch 9/200] [Batch 656/938] [D loss: 1.110697, acc: 96%] [G loss: 1.086614]
[Epoch 9/200] [Batch 657/938] [D loss: 1.102913, acc: 92%] [G loss: 1.105318]
[Epoch 9/200] [Batch 658/938] [D loss: 1.080597, acc: 99%] [G l

KeyboardInterrupt: 